In [1]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
beer_list = ["Heineken","Bud Light","Kronenbourg 1664","Newcastle Brown Ale","Rickard's Red","Kaiser Beer","Maclay's Traditional Pale Ale",
"Carlsberg","Weihenstephaner Hefeweissbier","Mill Street Original Organic Lager","Tatra Beer","Muskoka Cream Ale","Sinha Stout",
"Cool Beer Blonde Lager","Keller Premium Lager","Grolsch Premium Pilsner","Erdinger Weissbier","Beau's Lug Tread Lagered Ale",
"Czechvar Premium Lager","Great Lakes Brewery Canuck Pale Ale","DAB Original","Pilsner Urquell","Guinness Draught","Tsingtao Beer",
"Stella Artois", "Belgian Moon","Waterloo IPA","Goose Island IPA","Alexander Keith’s IPA","Kozel Černý (Dark)","Chang Beer",
"Kona Big Wave Golden Ale","Beck’s","Budwiser","Hoegaarden","Asahi","MacKinnon Brewing Red Fox Ale","Corona",
"Maisel","Krombacher Pils","Erdinger","Weihenstephan","Flying Monkeys 12 Minutes to Destiny",
"DAB", "Steam Whistle Premium Pilsner","Peroni","Tiger Beer","Moosehead Lager","Creemor","Lech Beer", 
"Collective Arts Ransack the Universe IPA","Cowbell Brewing Shindig Lager", "Nickel Brook Raspberry Uber Berliner",
"Belgian Moon Light Sky","Bavaria 8.6","Twisted Tea","Bud Light Apple","Bud Light Lime", "Boxer Ice","Carling Light",
"Estrella Dam","Czechvar","Dos Equis Lager","Dragon Stout","Harp Lager","Inis & Gunn Original", "Kirin Ichiban","Kozel",
"Mad Jack","Mickeys","Old Milwaukee","Michelob ultra","River Valley Golden Lager","La Fin Du Monde",
"Sapparo Premium", "Molson Canadian", "Ace Hill Lime Radler", "Mill Street 100th Meridian Amber","Side Launch Wheat",
"Toboggan Brewing Holy Roller Lager","Caledon Hills Helles","Flying Monkeys Worlds Away Modern Lager",
"Wellington Special Pale Ale","Kronenbourg 1664 Blanc","Smithwick's Ale","Nickel Brook Glory & Gold","Mill Street Watermelon Wheat",
"Sleeman Clear 2.0","Miller Lite","Shawn & Ed Brewing Co. LagerShed Original","Walkerville Kremlin Russian Imperial Stout",
"Muskoka Mad Tom IPA","Sleeman Point Five Citrus Lager","Nectarous", "Sartori Harvest IPA","Grande Cuvée Porter Baltique",
"Trois Pistoles","Bring Out Your Dead","Fat Tug IPA","Unibroue 17 Grande Réserve"]

In [3]:
len(beer_list)

100

In [4]:
beer_list = pd.DataFrame(data=beer_list, columns=['search_name'])
beer_list.head()

search_name
0             Heineken
1            Bud Light
2     Kronenbourg 1664
3  Newcastle Brown Ale
4        Rickard's Red

In [5]:
# Create DataFrame
data = pd.DataFrame(data=[], columns=['beer_info', 'search_name', 'beer_name'])
print(data)

# chromedriver.exe 
chromedriver = 'chromedriver.exe'
# URL
url = 'https://www.ratebeer.com/search?tab=beer'

# Web browser open using selenium
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(900, 900)
time.sleep(1)

Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []


In [6]:
def crawl(driver, beer, data, k):
    # Create Data Frame
    data = pd.DataFrame(data=[], columns=['beer_info', 'search_name', 'beer_name'])
    print(data)

    # url open
    print('url_open... {0} We are collecting the beer data..'.format(beer))
    
    
    
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)

    
    # Step1 : Search Beer
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    


    # Step2 : Select the Beer
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # Step3 : Scrapping the Product Beer
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # Step 4 : Scrapping all reviews (How many reviews do they have?)
            time.sleep(3)
            string = driver.find_element_by_class_name('MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # checking if ',' is included
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('Success!... get out of while loop.')
            break
        except :
            print('Error..! Restarts.')

            error_cnt += 1

            if error_cnt == 5:
                print('Error is keep showing... Move to the next beer!')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # Step 5 : Click Score breakdown 
    time.sleep(3)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # This is the code to move to the element. 
    driver.execute_script("arguments[0].click();", element)

    # calculate the number of the pages to check how many we need to scrap
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, 'Page')

        # Step 6 : scrapping all of the beer information
        time.sleep(3)
        beer_info = driver.find_elements_by_css_selector('.px-4.fj-s.f-wrap')

        tmp = []

        # save the data into dataframe 
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['search_name'] = beer
        data = pd.concat([data, tmp])

        # Step 7 : how to move to next page
        
        try :
            element = driver.find_element_by_xpath('//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('This is the last page')

    # cut the data by the number of the reviews if the data is duplicated
    if num != len(data):
        data = data[:num]

    print('The number of the reviews : ', num, 'The number of the reviews which are acutally collected : ', len(data))

    # Save as CSV, Excel 
    result = pd.merge(data, beer_list, on='search_name', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result




In [ ]:
# ForLoop
# for k in range(len(beer_list)):
#     result = crawl(driver, beer_list['search_name'].iloc[k], data, k)

In [ ]:
# ForLoop - 4
for k in range(25):
    k = k + 75
    result = crawl(driver, beer_list['search_name'].iloc[k], data, k)